# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

/Users/apple/miniforge3/envs/LLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the module from /Users/apple/.cache/huggingface/modules/datasets_modules/datasets/yelp_review_full/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf (last modified on Wed Dec 20 21:44:14 2023) since it couldn't be found locally at yelp_review_full., or remotely on the Hugging Face Hub.


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [4]:
dataset["train"][112]

{'label': 1,
 'text': "I'm not a huge fan of this location. I think that it was oddly built- the small, alley-like front makes it difficult to get past people on your way to/from the bathroom or tables when it's busy. And there's hardly any tables to sit at. Furthermore, people tend to clog the front on their way in, which makes things particularly difficult (especially in the winter weather). The staff were pretty impersonal to be, but maybe that was due to the high traffic of the place and the time I was there. And the coffee that I had was cold- I'm sure it was probably the bottom of the batch. I'd probably only walk in here again if someone else suggested it before or after a movie or while we were shopping in the area."}

In [5]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [6]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(dataset["train"])

,label,text
0,2 star,"Despite the decent atmosphere and not too terrible location we get sat at one of the greasiest tables I've seen in a long time. We order appetizers and they come posthaste. Good, but not worth 10$ a plate good... Then comes the pink prickly pear chicken :/ poor execution at a great recipe. The goat cheese bruschetta was OK but slightly disappointing. If you want a 50$ tab for a few drink+ appetizers and 1 entree i suggest you go somewhere else. Oh and don't even think about giving them a bad yelp review, the owner will put a plaque on the wall for anyone that takes the time to write about how disappointing the food is. Guy can't take a hint."
1,3 stars,"We were looking for an outdoor dining on a Sunday evening and found Nori Sushi at the Market Place. When we arrived we were promptly seated with the menu; however, after waiting for a server for a while I had to ask a busboy to send someone to us. The place was pretty empty at the time,\n\nStarted off with fried calamari, which was tasty but they were very tiny. Good miso soup. Ahi poke was fresh and had good flavor. Spicy salmon was very good. Hamachi sashimi was okay. Good service."
2,5 stars,I am SO in love with the banana split sandwich!! Great food. Great service. Indoor and outdoor seating.
3,1 star,"No stars! Horrible I just wanted tap water after my husband bought a $4 coffee. Crystal refused and laughed and said come back with your own cup so I did and they all laughed l. I asked for a manager and she gave me a card that said \""princess rippy\"" (702) 497-9598 horrible horrible"
4,1 star,"The name says it all, culinary dropout, food and service both worthy of expulsion. \n\nI have never been so disappointed or disrespected in a restaurant as I was at culinary dropout. I placed an order for pickup and when I arrived, the food was sitting by the cash register at the bar in the open. It wasn't under a heat lamp or in anything to keep it warm, I even made a comment to the rude bartender who wanted to finish drying his wine glasses before approaching me asking why the food wasn't being kept warm and that it would be cold before getting home and his response was that it was warm. Then he printed out the check and all he said was \""cheers!\"" \n\nI ordered the steak, fish and chips and soft pretzel, all of which were cold and soggy, the steak was already cut up and half of it was fat. Not just marbled with fat, pure yellow fat! The fries were soggy, fondue dipping sauce was solidified, fish and chips were cold and limp. \n\nGiving the restaurant one star is generous, they can bet their bottom dollar I won't be returning,"
5,4 stars,"Wow, one of the best filets in Phoenix. I was skeptical but VERY impressed. You can pay extra for some steak toppings such as lump crab meat or shrimp, but be forewarned those sides are miniscule. Not worth the price - stick with just the filet and the sides it comes with. Plenty of food. Be sure to order to pretzel bread with cheese as an appetizer!"
6,5 stars,"I love coming here. The guys let me mix the flavors & actually add shots of vodka for no additional cost!! These guys are nice too. Its worth the prices. When visiting Vegas, come to this\""Evening Call\"" location. My favorite..."
7,1 star,"DO NOT STAY IN THIS HOTEL IF YOU ARE OVER 25 YEARS OF AGE!\n\nI stayed at the Planet Hollywood Hotel on April 18-21. It was the most miserable hotel experience that I have ever had. \n\nFirstly, when I took the bus from the airport, the busses are instructed to drop guests off at a certain location. It was a long walk from that point to the hotel reception (on an underground street), and there was no one to assist with bags. I was traveling alone with a suitcase, a carry on, a computer bag and a infant car seat (meeting my family later), and I had to manage all this by myself. The hotel itself has double doors at the entrance, and they are neither revolving nor automated. And there is no doorman to open the door for you.\n\nI ha

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [9]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,1 star,Horrible sushi.... service was fine but will not be back!,"[101, 9800, 27788, 28117, 5933, 119, 119, 119, 119, 1555, 1108, 2503, 1133, 1209, 1136, 1129, 1171, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [10]:
#change with full data set
#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
#small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
full_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)


## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [12]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [13]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_re

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [14]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Using the latest cached version of the module from /Users/apple/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Wed Dec 20 22:12:18 2023) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.



接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)

## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:

```shell
Every 1.0s: nvidia-smi                                                   Wed Dec 20 14:37:41 2023

Wed Dec 20 14:37:41 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   64C    P0              69W /  70W |   6665MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     18395      C   /root/miniconda3/bin/python                6660MiB |
+---------------------------------------------------------------------------------------+
```

In [18]:
trainer.train()

/Users/apple/miniforge3/envs/LLM/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 30/121875 [00:34<37:55:20,  1.12s/it]

{'loss': 1.5949, 'learning_rate': 4.998769230769231e-05, 'epoch': 0.0}


  0%|          | 60/121875 [01:08<37:37:19,  1.11s/it]

{'loss': 1.3109, 'learning_rate': 4.997538461538462e-05, 'epoch': 0.0}


  0%|          | 90/121875 [01:41<37:33:15,  1.11s/it]

...

{'loss': 0.6242, 'learning_rate': 9.606153846153847e-06, 'epoch': 2.42}


 81%|████████  | 98490/121875 [30:52:21<7:22:03,  1.13s/it]

{'loss': 0.6068, 'learning_rate': 9.593846153846154e-06, 'epoch': 2.42}


 81%|████████  | 98500/121875 [30:52:32<7:16:47,  1.12s/it]

RuntimeError: [enforce fail at inline_container.cc:424] . unexpected pos 267843008 vs 267842896

In [19]:
#small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))
full_test_dataset = tokenized_datasets["test"].shuffle(seed=64)

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


In [22]:
trainer.evaluate(full_test_dataset)

                                                           
 81%|████████  | 98500/121875 [31:37:51<7:16:47,  1.12s/it]

{'eval_loss': 0.7419986724853516, 'eval_accuracy': 0.68586, 'eval_runtime': 941.4775, 'eval_samples_per_second': 53.108, 'eval_steps_per_second': 6.639, 'epoch': 2.42}


{'eval_loss': 0.7419986724853516,
 'eval_accuracy': 0.68586,
 'eval_runtime': 941.4775,
 'eval_samples_per_second': 53.108,
 'eval_steps_per_second': 6.639,
 'epoch': 2.42}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [23]:
trainer.save_model(model_dir)

In [21]:
trainer.save_state()

In [23]:
# trainer.model.save_pretrained("./")

## Homework: 使用完整的 YelpReviewFull 数据集训练，看 Acc 最高能到多少